In [12]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.font_manager as fm
import seaborn as sns
import statsmodels.api as sm

font_path = 'C:\\Windows\\Fonts\\gulim.ttc'
font = fm.FontProperties(fname=font_path).get_name()
mpl.rc('font', family=font)

In [13]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

def divide_time_range(time):
    if time < 600:
        return '00:00~06:00'
    elif time < 1000:
        return '06:00~10:00' # 출근시간
    elif time < 1700:
        return '10:00~17:00'
    elif time < 2100:
        return '17:00~21:00' # 퇴근시간
    else:
        return '21:00~24:00'

def check_best_arima(data):
    best_aic = float("inf")  # 초기값 설정
    best_bic = float("inf")  # 초기값 설정
    best_model = None
    best_p, best_d, best_q = None, None, None
    
    for p in range(1, 6):
        for d in range(1, 3):
            for q in range(0, 6):
                try:
                    model = ARIMA(data, order=(p, d, q))
                    model_fit = model.fit()
                    aic = model_fit.aic
                    bic = model_fit.bic
                    if bic < best_bic or (bic == best_bic and aic < best_aic):
                        best_aic = aic
                        best_bic = bic
                        best_model = model_fit
                        best_p, best_d, best_q = p, d, q
                except:
                    continue
    
    return best_model, best_p, best_d, best_q

def arima_test(data, IDs):
    for 주거_ID in IDs:
        주거_data = data[data['대여소_ID'] == 주거_ID]
        주거_data['기준_날짜'] = pd.to_datetime(주거_data['기준_날짜'])
        주거_data.set_index('기준_날짜', inplace=True)
        arima_data = 주거_data.resample('D')['전체_건수'].sum()
        
        best_model, best_p, best_d, best_q = check_best_arima(arima_data)
        if best_model is not None:
            print(f"대여소 {주거_ID}의 best model (p, d, q): ({best_p}, {best_d}, {best_q})")
            forecast = best_model.forecast(steps=1)  # 다음 시간대에 대한 예측값
            print(f"대여소 {주거_ID}의 다음 시간대 예측값: {forecast}")



KeyboardInterrupt: 

In [15]:
강서_대여소_ID = pd.read_csv('../../Data/따릉이/강서_따릉이_대여소_ID.csv',encoding='utf-8')

강서_데이터1 = pd.read_csv('../../Data/따릉이/강서_따릉이_데이터_01.csv',encoding='utf-8')
강서_데이터2 = pd.read_csv('../../Data/따릉이/강서_따릉이_데이터_02.csv',encoding='utf-8')

강서_데이터 = pd.concat([강서_데이터1, 강서_데이터2], ignore_index=True)
print(len(강서_데이터))

top_20_ID = pd.read_csv('../../Data/따릉이/쏠림정도_top20_대여소.csv',encoding='utf-8')
# 주거와 상업으로 데이터 분할
주거_df = top_20_ID[top_20_ID['최종용도'] == '주거']
상업_df = top_20_ID[top_20_ID['최종용도'] == '상업']

# 주거 데이터에서 대여소_ID 추출
주거_IDs = 주거_df['대여소_ID'].unique()

# 상업 데이터에서 대여소_ID 추출
상업_IDs = 상업_df['대여소_ID'].unique()

강서_df = 강서_데이터[강서_데이터['종료_대여소_ID'].isin(주거_IDs)] # 시작 : 대여 ,종료 : 반납   + 주거, 상업
강서_df = 강서_df.sort_values(by='기준_날짜')

강서_df['시간대_구간'] = 강서_df['기준_시간대'].apply(divide_time_range)
강서_df1 = 강서_df[강서_df['시간대_구간'] == '17:00~21:00']     # 06~10 : , 17:21 :퇴근
강서_df1 = 강서_df1[(강서_df1['전체_이용_분'] != 0) & (강서_df1['전체_이용_거리'] != 0)]

강서_df1 = 강서_df1[['종료_대여소_ID', '기준_날짜', '전체_건수']].copy() # 시작 : 대여 ,종료 : 반납
강서_df1 = 강서_df1.rename(columns={'종료_대여소_ID': '대여소_ID'})     # 시작 : 대여 ,종료 : 반납

from statsmodels.tsa.stattools import adfuller

# ADF 테스트 수행
adf_result = adfuller(강서_df1['전체_건수'])

# ADF 통계량 및 p-value 출력
print(f'ADF 통계량: {adf_result}')
print(f'p-value: {adf_result[1]}')

# 결과 해석
if adf_result[1] < 0.05:
    print('p-value가 유의수준보다 작으므로 귀무가설을 기각합니다. 시계열 데이터는 정상성을 만족합니다.')
else:
    print('p-value가 유의수준보다 크거나 같으므로 귀무가설을 기각할 수 없습니다. 시계열 데이터는 정상성을 만족하지 않을 수 있습니다.')
#arima_test(강서_df1,주거_IDs) # 주거 or 상업


# 출근, 대여   :  성원님  주거, 상업,
# 출근, 반납   :  동완님  
# 퇴근, 대여   :  율희님
# 퇴근, 반납   :  저

3385613
ADF 통계량: (-40.46574950648836, 0.0, 42, 93706, {'1%': -3.4304197871976583, '5%': -2.8615708448251813, '10%': -2.5667864176250905}, 3940.8495127995848)
p-value: 0.0
p-value가 유의수준보다 작으므로 귀무가설을 기각합니다. 시계열 데이터는 정상성을 만족합니다.


In [10]:
강서_대여소_ID = pd.read_csv('../../Data/따릉이/강서_따릉이_대여소_ID.csv',encoding='utf-8')

강서_데이터1 = pd.read_csv('../../Data/따릉이/강서_따릉이_데이터_01.csv',encoding='utf-8')
강서_데이터2 = pd.read_csv('../../Data/따릉이/강서_따릉이_데이터_02.csv',encoding='utf-8')

강서_데이터 = pd.concat([강서_데이터1, 강서_데이터2], ignore_index=True)
print(len(강서_데이터))

top_20_ID = pd.read_csv('../../Data/따릉이/쏠림정도_top20_대여소.csv',encoding='utf-8')
# 주거와 상업으로 데이터 분할
주거_df = top_20_ID[top_20_ID['최종용도'] == '주거']
상업_df = top_20_ID[top_20_ID['최종용도'] == '상업']

# 주거 데이터에서 대여소_ID 추출
주거_IDs = 주거_df['대여소_ID'].unique()

# 상업 데이터에서 대여소_ID 추출
상업_IDs = 상업_df['대여소_ID'].unique()

강서_df = 강서_데이터[강서_데이터['종료_대여소_ID'].isin(상업_IDs)] # 시작 : 대여 ,종료 : 반납   + 주거, 상업
강서_df = 강서_df.sort_values(by='기준_날짜')

강서_df['시간대_구간'] = 강서_df['기준_시간대'].apply(divide_time_range)
강서_df1 = 강서_df[강서_df['시간대_구간'] == '17:00~21:00']     # 06~10 : , 17:21 :퇴근
강서_df1 = 강서_df1[(강서_df1['전체_이용_분'] != 0) & (강서_df1['전체_이용_거리'] != 0)]

강서_df1 = 강서_df1[['종료_대여소_ID', '기준_날짜', '전체_건수']].copy() # 시작 : 대여 ,종료 : 반납
강서_df1 = 강서_df1.rename(columns={'종료_대여소_ID': '대여소_ID'})     # 시작 : 대여 ,종료 : 반납

arima_test(강서_df1,상업_IDs) # 주거 or 상업


# 출근, 대여   :  성원님  주거, 상업,
# 출근, 반납   :  동완님
# 퇴근, 대여   :  율희님
# 퇴근, 반납   :  저

3385613


C:\Users\ciw96\AppData\Local\Temp\ipykernel_41812\3542165546.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  주거_data['기준_날짜'] = pd.to_datetime(주거_data['기준_날짜'])
c:\Users\ciw96\AppData\Local\Programs\Python\Python310\lib\site-packages\statsmodels\tsa\statespace\sarimax.py:978: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting parameters.
  warn('Non-invertible starting MA parameters found.'
c:\Users\ciw96\AppData\Local\Programs\Python\Python310\lib\site-packages\statsmodels\tsa\statespace\sarimax.py:978: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting parameters.
  warn('Non-invertible starting MA parameters found.'
c:\Users\ciw96\AppData\Local\Programs\Python\Python310\lib\sit

대여소 ST-1064의 best model (p, d, q): (5, 1, 5)
대여소 ST-1064의 다음 시간대 예측값: 2024-03-30    6.937452
Freq: D, dtype: float64


c:\Users\ciw96\AppData\Local\Programs\Python\Python310\lib\site-packages\statsmodels\tsa\statespace\sarimax.py:966: UserWarning: Non-stationary starting autoregressive parameters found. Using zeros as starting parameters.
  warn('Non-stationary starting autoregressive parameters'
c:\Users\ciw96\AppData\Local\Programs\Python\Python310\lib\site-packages\statsmodels\tsa\statespace\sarimax.py:978: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting parameters.
  warn('Non-invertible starting MA parameters found.'
c:\Users\ciw96\AppData\Local\Programs\Python\Python310\lib\site-packages\statsmodels\tsa\statespace\sarimax.py:978: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting parameters.
  warn('Non-invertible starting MA parameters found.'
c:\Users\ciw96\AppData\Local\Programs\Python\Python310\lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retva

대여소 ST-1688의 best model (p, d, q): (4, 1, 3)
대여소 ST-1688의 다음 시간대 예측값: 2024-03-30    3.553334
Freq: D, dtype: float64


...

c:\Users\ciw96\AppData\Local\Programs\Python\Python310\lib\site-packages\statsmodels\tsa\statespace\sarimax.py:978: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting parameters.
  warn('Non-invertible starting MA parameters found.'
c:\Users\ciw96\AppData\Local\Programs\Python\Python310\lib\site-packages\statsmodels\tsa\statespace\sarimax.py:966: UserWarning: Non-stationary starting autoregressive parameters found. Using zeros as starting parameters.
  warn('Non-stationary starting autoregressive parameters'
c:\Users\ciw96\AppData\Local\Programs\Python\Python310\lib\site-packages\statsmodels\tsa\statespace\sarimax.py:978: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting parameters.
  warn('Non-invertible starting MA parameters found.'
c:\Users\ciw96\AppData\Local\Programs\Python\Python310\lib\site-packages\statsmodels\tsa\statespace\sarimax.py:978: UserWarning: Non-invertible starting MA parameters found. Using zeros as st

대여소 ST-3128의 best model (p, d, q): (2, 1, 3)
대여소 ST-3128의 다음 시간대 예측값: 2024-03-30    1.52338
Freq: D, dtype: float64


c:\Users\ciw96\AppData\Local\Programs\Python\Python310\lib\site-packages\statsmodels\tsa\statespace\sarimax.py:978: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting parameters.
  warn('Non-invertible starting MA parameters found.'
c:\Users\ciw96\AppData\Local\Programs\Python\Python310\lib\site-packages\statsmodels\tsa\statespace\sarimax.py:966: UserWarning: Non-stationary starting autoregressive parameters found. Using zeros as starting parameters.
  warn('Non-stationary starting autoregressive parameters'
c:\Users\ciw96\AppData\Local\Programs\Python\Python310\lib\site-packages\statsmodels\tsa\statespace\sarimax.py:978: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting parameters.
  warn('Non-invertible starting MA parameters found.'
c:\Users\ciw96\AppData\Local\Programs\Python\Python310\lib\site-packages\statsmodels\tsa\statespace\sarimax.py:978: UserWarning: Non-invertible starting MA parameters found. Using zeros as st

대여소 ST-3251의 best model (p, d, q): (2, 1, 3)
대여소 ST-3251의 다음 시간대 예측값: 2024-03-30    1.286065
Freq: D, dtype: float64


c:\Users\ciw96\AppData\Local\Programs\Python\Python310\lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
